In [12]:
import requests

base_url = 'http://127.0.0.1:5000/uppercase'
params= {'text': 'Hello Keith'}
response = requests.get(base_url, params=params)

# Print the JSON content from the response
if response.status_code == 200:
    print(response.json())  # Call the method with parentheses
else:
    print(f"Request failed with status code {response.status_code}")

{'text': 'HELLO KEITH'}
